# Create Boundary Conditions of the 79NG Fjord GETM Setup

This notebook creates the GETM input files describing the boundary conditions of the 79NG fjord setup.
The files for the corresponding initial conditions are also created.

Notebook by Markus Reinert (IOW, 2023–2024, https://orcid.org/0000-0002-3761-8029).

In [ ]:
from datetime import datetime

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from tools.configuration import Configuration

In [ ]:
config = Configuration()

## Load the topography

In [ ]:
filename = config.get_file_path("getm/domain/bathymetry")
print(f"Loading topography from {filename!r}.")
topo = xr.open_dataset(filename)
topo

## Show the land/ocean mask

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6), dpi=300)
ax.pcolormesh(topo.mask)
ax.set_title("GETM index 1 is between grid lines 0 and 1\nGETM index 11 is above/to the right of grid line 10")
ax.set_aspect("equal")
ax.set_xticks(np.arange(0, topo.lon.size + 1, 10))
ax.set_yticks(np.arange(0, topo.lat.size + 1, 10))
ax.set_xticks(np.arange(topo.lon.size + 1), minor=True)
ax.set_yticks(np.arange(topo.lat.size + 1), minor=True)
ax.grid(which="minor", linewidth=0.5)
ax.grid(color="red", linewidth=0.5)

## Create the boundary info file

The file format is explained at https://getm.eu/bdys/articles/bdys.html.
Here's a compass for orientation:
|     |     |     |
| :-: | :-: | :-: |
| NW  |**N**| NE  |
|**W**|  ↑  |**E**|
| SW  |**S**| SE  |

### Get start and end indices of the open boundaries

In [ ]:
# North
# From the first ocean point (mask is True) to one before the last grid point,
# because the last grid point (NE corner) belongs to the eastern boundary
i_bdy_start_N = np.where(topo.mask.isel(lat=-1))[0][0]
i_bdy_end_N = topo.lon.size - 2

# East
# Full extent of the model grid
i_bdy_start_E = 0
i_bdy_end_E = topo.lat.size - 1

# South
# Section 1: from the first ocean point to the island
i_bdy_start_S1 = np.where(topo.mask.isel(lat=0))[0][0]
i_bdy_end_S1 = np.where(~topo.mask.isel(lat=0, lon=slice(i_bdy_start_S1, None)))[0][0] + i_bdy_start_S1 - 1
# Section 2: from the island to one before the last grid point,
# because the last grid point (SE corner) belongs to the eastern boundary
i_bdy_start_S2 = np.where(topo.mask.isel(lat=0, lon=slice(i_bdy_end_S1 + 1, None)))[0][0] + i_bdy_end_S1 + 1
i_bdy_end_S2 = topo.lon.size - 2

In [ ]:
n_bdy_points = (
    i_bdy_end_N  - i_bdy_start_N  + 1 +
    i_bdy_end_E  - i_bdy_start_E  + 1 + 
    i_bdy_end_S1 - i_bdy_start_S1 + 1 +
    i_bdy_end_S2 - i_bdy_start_S2 + 1
)
print("Open boundary consists of", n_bdy_points, "grid points.")

### Create the file content

Add 1 to every index, because GETM starts counting indices at 1 and not 0.

In [ ]:
boundary_info = f"""\
# no western boundary
0
# northern boundary
1
{topo.lat.size} {i_bdy_start_N + 1} {i_bdy_end_N + 1} 3 0
# eastern boundary over the full model domain
1
{topo.lon.size} {i_bdy_start_E + 1} {i_bdy_end_E + 1} 3 0
# southern boundaries
2
1 {i_bdy_start_S1 + 1} {i_bdy_end_S1 + 1} 3 0
1 {i_bdy_start_S2 + 1} {i_bdy_end_S2 + 1} 3 0
"""
print(boundary_info.strip())

### Save the file

In [ ]:
filename = config.get_file_path("getm/domain/bdyinfofile")
with open(filename, "w") as f:
    f.write(boundary_info)
print(f"Saved the boundary info as {filename!r}.")

## Create 2D boudary conditions

### Create the dataset

In [ ]:
time_string_0 = config.get_text("getm/time/start")
time_string_1 = config.get_text("getm/time/stop")
datetime_0 = datetime.strptime(time_string_0, "%Y-%m-%d %H:%M:%S")
datetime_1 = datetime.strptime(time_string_1, "%Y-%m-%d %H:%M:%S")
print(f"Model runs from {datetime_0} to {datetime_1}.")

In [ ]:
bdy_2D = xr.Dataset(
    {
        "elev": (
            ["time", "nbdy"],
            np.zeros((2, n_bdy_points)),
            {"long_name": "Sea surface elevation", "units": "m"},
        ),
    },
    coords={
        "nbdy": (["nbdy"], np.arange(n_bdy_points)),
        "time": (["time"], [datetime_0, datetime_1]),
    },
    attrs={
        "title": "Boundary conditions (2D) for the 79NG fjord GETM setup",
        "author": "Markus Reinert (ORCID: 0000-0002-3761-8029)",
        "institution": "Leibniz Institute for Baltic Sea Research Warnemuende (IOW), Germany",
        "description": "Boundary conditions are given by constant zero elevation.",
    },
)
bdy_2D

### Save the dataset

In [ ]:
filename = config.get_file_path("getm/m2d/bdyfile_2d")
bdy_2D.to_netcdf(
    filename,
    unlimited_dims=["time"],
    encoding={
        "elev": {"_FillValue": None},
        "time": {"units": "seconds since 2000-01-01"},
    },
)
print(f"Saved the 2D boundary conditions as {filename!r}.")

## Create 3D boundary conditions

### Get the stratification

The stratification is defined by layers of given temperatures ($T$) and salinities ($S$) at given depths ($-z$).
Between the given $z$-levels, $T$ and $S$ are linearly interpolated.

In [ ]:
stratification = []
for level in config.get_element("fjord/stratification"):
    assert level.tag == "level",\
        f"all elements in stratification must be named 'level', not {level.tag!r}"
    stratification.append([float(level.get(var)) for var in "zST"])
stratification.sort(key=lambda level: level[0], reverse=True)
z_discrete, S_discrete, T_discrete = zip(*stratification)

### Show the stratification

In [ ]:
print(f"  {'z':^5}   {'S':^5}   {'T':^5}")
print("-" * 24)
print(np.array(stratification))

fig, axs = plt.subplots(ncols=2, sharey=True, constrained_layout=True)
fig.suptitle("Stratification")
axs[0].plot(S_discrete, z_discrete)
axs[1].plot(T_discrete, z_discrete)
axs[0].set_xlabel("Salinity $S$ [g/kg]")
axs[1].set_xlabel("Temperature $T$ [°C]")
axs[0].set_ylabel("Vertical coordinate $z$ [m]")
for ax in axs:
    ax.grid()

### Create the dataset

In [ ]:
bdy_3D = xr.Dataset(
    {
        "salt": (
            ["time", "nbdy", "zax"],
            np.ones((2, n_bdy_points, len(z_discrete))) * S_discrete,
            {"long_name": "Salinity", "units": "g/kg"},
        ),
        "temp": (
            ["time", "nbdy", "zax"],
            np.ones((2, n_bdy_points, len(z_discrete))) * T_discrete,
            {"long_name": "Temperature", "units": "degC"},
        ),
    },
    coords={
        "nbdy": (["nbdy"], bdy_2D.nbdy.data),
        "time": (["time"], bdy_2D.time.data),
        "zax": (["zax"], list(z_discrete), {"long_name": "z-axis", "units": "m", "positive": "up"}),
    },
    attrs={
        "title": "Boundary conditions (3D) for the 79NG fjord GETM setup",
        "author": bdy_2D.author,
        "institution": bdy_2D.institution,
        "description": "Boundary conditions are given by piece-wise linear z-profiles of S and T.",
    },
)
bdy_3D

### Save the dataset

In [ ]:
filename = config.get_file_path("getm/m3d/bdyfile_3d")
bdy_3D.to_netcdf(
    filename,
    unlimited_dims=["time"],
    encoding={
        "zax": {"_FillValue": None},
        "salt": {"_FillValue": None},
        "temp": {"_FillValue": None},
        "time": {"units": "seconds since 2000-01-01"},
    },
)
print(f"Saved the 3D boundary conditions as {filename!r}.")

### Create corresponding initial conditions

#### Salinity profile

In [ ]:
filename = config.get_file_path("getm/salt/salt_file")
with open(filename, "w") as f:
    f.write(f"{len(z_discrete)}\n")
    for z_val, S_val in zip(z_discrete, S_discrete):
        f.write(f"{z_val:6} {S_val:3}\n")
print(f"Saved initial salinity stratification as {filename!r}.")

#### Temperature profile

In [ ]:
filename = config.get_file_path("getm/temp/temp_file")
with open(filename, "w") as f:
    f.write(f"{len(z_discrete)}\n")
    for z_val, T_val in zip(z_discrete, T_discrete):
        f.write(f"{z_val:6} {T_val:5}\n")
print(f"Saved initial temperature stratification as {filename!r}.")